In [9]:
import pyarrow as pa
import pyarrow.parquet as pq
from pgpq import ArrowToPostgresBinaryEncoder

In [14]:
arrow_table = pq.read_table("tests/testdata/yellow_tripdata_2022-01.parquet")
encoder = ArrowToPostgresBinaryEncoder(arrow_table.schema)

full_file_batches = arrow_table.to_batches()
# use just the first row until https://github.com/apache/arrow-rs/issues/3646 gets resolved
batch = pa.RecordBatch.from_pylist(full_file_batches[0].slice(0, 1).to_pylist())
small_batches = [pa.RecordBatch.from_pylist(batch.to_pylist() * 100)]
large_batches = [pa.RecordBatch.from_pylist(batch.to_pylist() * 10_000) for _ in range(10_000)]  # 10k batches of 10k rows = 100m

In [15]:
def encode(batches) -> None:
    for i, batch in enumerate(batches):
        try:
            encoder.encode(batch)
        except Exception:
            print(f"Exception in batch {i}")
            raise

In [16]:
%timeit encode(small_batches)

684 µs ± 3.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
from time import time

start = time()
encode(large_batches)
end = time()
print(f"{end-start}")

592.5959279537201
